# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 20.09.2022

Мягкий дедлайн: 23:59MSK 04.10.2022

Жесткий дедлайн: 23:59MSK 10.10.2022

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install wandb --upgrade --quiet
import wandb

wandb.login()
wandb.init(project="iad_homework1")

     |████████████████████████████████| 1.8 MB 30.1 MB/s 
     |████████████████████████████████| 162 kB 61.0 MB/s 
     |████████████████████████████████| 181 kB 48.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 162 kB 32.1 MB/s 
     |████████████████████████████████| 158 kB 54.3 MB/s 
     |████████████████████████████████| 157 kB 62.2 MB/s 
     |████████████████████████████████| 157 kB 56.5 MB/s 
     |████████████████████████████████| 157 kB 50.9 MB/s 
     |████████████████████████████████| 157 kB 73.5 MB/s 
     |████████████████████████████████| 157 kB 64.6 MB/s 
     |████████████████████████████████| 157 kB 63.1 MB/s 
     |████████████████████████████████| 157 kB 55.1 MB/s 
     |████████████████████████████████| 156 kB 55.5 MB/s 


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgkomarov_hse. Use `wandb login --relogin` to force relogin


In [3]:
# %conda install wget

In [4]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2022-10-04 15:34:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211011981 (201M) [application/x-httpd-php]
Saving to: ‘data.txt.zip’

data.txt.zip        100%[===================>] 201.24M  41.9MB/s    in 5.3s    

2022-10-04 15:34:15 (38.3 MB/s) - ‘data.txt.zip’ saved [211011981/211011981]



In [5]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [6]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [7]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [8]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

print(f'RMSE Ridge регресии на тестовой выборке: {mean_squared_error(y_test, y_pred)**0.5}')

RMSE Ridge регресии на тестовой выборке: 9.510160711373397


In [9]:
# Самый лучший константный прогноз - среднее 
y_pred_const = len(y_test) * [y_test.mean()]
print(f'RMSE при константном прогнозе на тестовой выборке: {mean_squared_error(y_test, y_pred_const)**0.5}')

RMSE при константном прогнозе на тестовой выборке: 10.851909820717683


## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [10]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [11]:
set_random_seed(1)

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [12]:
# Функция из семинара
class RandomDataset(torch.utils.data.Dataset):
    """
    Our random dataset
    """
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return {"sample": torch.tensor(self.x[idx, :], dtype=torch.float), "target": self.y[idx]}

### Experiment 1

In [13]:
train_set = RandomDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True)

test_set = RandomDataset(X_test, y_test) 
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=True)

model1 = nn.Sequential(
    nn.Linear(90, 65),
    nn.ReLU(),
    nn.Linear(65, 40),
    nn.ReLU(),
    nn.Linear(40, 1)
)
wandb.watch(model1);

optimizer1 = torch.optim.Adam(model1.parameters())
criterion = nn.MSELoss()
epoch = 15

In [14]:
from numpy.random import logseries
def train(model, optimizer, criterion, train_loader, test_loader, epoch):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''

    for i in range(epoch):
        train_loss, test_loss = [], []

        for batch in train_loader:  
            y_pred = model(batch['sample'])                      
            loss = criterion(y_pred, batch['target'].float())

            loss.backward()                         
            optimizer.step()                             
            optimizer.zero_grad()

            train_loss.append(np.sqrt(loss.detach().numpy()))

        
        with torch.no_grad():                         
            for batch in test_loader: 
                y_pred = model(batch['sample']) 
                loss = criterion(y_pred, batch['target'].float())              
                 
                test_loss.append(np.sqrt(loss.numpy()))

        wandb.log({"mean train loss": np.mean(train_loss),
                   "mean test loss": np.mean(test_loss)})

        print(f'Epoch: {i+1}\n\tTrain loss: {np.mean(train_loss)}\n\tTest loss: {np.mean(test_loss)}')


def test(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    model.eval()
    test_loss, predicts = [], []

    with torch.no_grad():                         
        for batch in test_loader: 
          y_pred = model(batch['sample']) 
          loss = criterion(y_pred, batch['target'].float()) 

          test_loss.append(np.sqrt(loss.numpy()))
          predicts.extend(y_pred.squeeze(1).tolist())

    return {"predicts": torch.tensor(predicts), "RMSE": np.mean(test_loss)}

In [15]:
assert test(model1, criterion, test_loader)['predicts'].shape[0] == y_test.shape[0]

In [16]:
train(model1, optimizer1, criterion, train_loader, test_loader, epoch)

Epoch: 1
	Train loss: 323.4642028808594
	Test loss: 164.33721923828125
Epoch: 2
	Train loss: 147.168212890625
	Test loss: 139.9851837158203
Epoch: 3
	Train loss: 135.72320556640625
	Test loss: 125.32875061035156
Epoch: 4
	Train loss: 129.3109130859375
	Test loss: 126.22613525390625
Epoch: 5
	Train loss: 123.52549743652344
	Test loss: 116.8316421508789
Epoch: 6
	Train loss: 120.8100357055664
	Test loss: 116.00330352783203
Epoch: 7
	Train loss: 117.46675109863281
	Test loss: 150.123046875
Epoch: 8
	Train loss: 115.79367065429688
	Test loss: 124.0644302368164
Epoch: 9
	Train loss: 113.31788635253906
	Test loss: 107.8038101196289
Epoch: 10
	Train loss: 112.42581939697266
	Test loss: 108.51280212402344
Epoch: 11
	Train loss: 110.64624786376953
	Test loss: 110.4381103515625
Epoch: 12
	Train loss: 109.65849304199219
	Test loss: 113.22065734863281
Epoch: 13
	Train loss: 108.49878692626953
	Test loss: 116.21568298339844
Epoch: 14
	Train loss: 107.01498413085938
	Test loss: 101.59498596191406
Ep

### Experiment 2

In [17]:
wandb.login()
wandb.init(project="iad_homework1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


mean test loss,█▅▄▄▃▃▆▄▂▂▂▂▃▁▁
mean train loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁
mean test loss,101.42126
mean train loss,105.79577


In [18]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler()
X_train_norm = x_scaler.fit_transform(X_train)
X_test_norm = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_norm = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_norm = y_scaler.transform(y_test.reshape(-1, 1))

In [19]:
train_set_norm = RandomDataset(X_train_norm, y_train_norm)
train_loader_norm = torch.utils.data.DataLoader(train_set_norm, batch_size=256, shuffle=True)

test_set_norm = RandomDataset(X_test_norm, y_test_norm) 
test_loader_norm = torch.utils.data.DataLoader(test_set_norm, batch_size=256, shuffle=True)

model2 = nn.Sequential(
    nn.Linear(90, 65),
    nn.ReLU(),
    nn.Linear(65, 40),
    nn.ReLU(),
    nn.Linear(40, 1)
)
wandb.watch(model2);

optimizer2 = torch.optim.Adam(model2.parameters())

In [20]:
def train_norm(model, optimizer, criterion, train_loader, test_loader, epoch, save=False):

    min_loss = float('inf')
    for i in range(epoch):
        train_loss, test_loss = [], []

        for batch in train_loader:  
            y_pred = model(batch['sample'])                      
            loss = criterion(y_pred, batch['target'].float())

            loss.backward()                         
            optimizer.step()                             
            optimizer.zero_grad()

            loss_train = criterion(torch.tensor(y_scaler.inverse_transform(y_pred.detach().numpy().reshape(-1,1)), requires_grad=False).float(),
                                   torch.tensor(y_scaler.inverse_transform(batch['target'].reshape(-1,1)), requires_grad=False).float())

            train_loss.append(np.sqrt(loss_train))

        
        with torch.no_grad():                         
            for batch in test_loader: 
                y_pred = model(batch['sample']) 
                loss_test = criterion(torch.tensor(y_scaler.inverse_transform(y_pred.detach().numpy().reshape(-1,1))).float(),
                                      torch.tensor(y_scaler.inverse_transform(batch['target'].reshape(-1,1))).float())
                 
                test_loss.append(np.sqrt(loss_test))
                
        if (save == True) and (min_loss > np.mean(test_loss)):
            min_loss = np.mean(test_loss)
            torch.save(model.state_dict(), 'best_model.pt')

        wandb.log({"mean train loss": np.mean(train_loss),
                   "mean test loss": np.mean(test_loss)})

        print(f'Epoch: {i+1}\n\tTrain loss: {np.mean(train_loss)}\n\tTest loss: {np.mean(test_loss)}')


def test_norm(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    model.eval()
    test_loss, predicts = [], []

    with torch.no_grad():                         
        for batch in test_loader: 
          y_pred = model(batch['sample']) 
          loss_test = criterion(torch.tensor(y_scaler.inverse_transform(y_pred.detach().numpy().reshape(-1,1))).float(),
                                torch.tensor(y_scaler.inverse_transform(batch['target'].reshape(-1,1))).float())

          test_loss.append(np.sqrt(loss_test))
          predicts.extend(y_pred.squeeze(1).tolist())
          
    return {"predicts": torch.tensor(predicts), "RMSE": np.mean(test_loss)}

In [21]:
train_norm(model2, optimizer2, criterion, train_loader_norm, test_loader_norm, epoch)

Epoch: 1
	Train loss: 8.971867561340332
	Test loss: 8.859606742858887
Epoch: 2
	Train loss: 8.69029426574707
	Test loss: 8.820600509643555
Epoch: 3
	Train loss: 8.612788200378418
	Test loss: 8.78053092956543
Epoch: 4
	Train loss: 8.567011833190918
	Test loss: 8.831060409545898
Epoch: 5
	Train loss: 8.526538848876953
	Test loss: 8.807101249694824
Epoch: 6
	Train loss: 8.493990898132324
	Test loss: 8.792665481567383
Epoch: 7
	Train loss: 8.467588424682617
	Test loss: 8.802465438842773
Epoch: 8
	Train loss: 8.449584007263184
	Test loss: 8.843875885009766
Epoch: 9
	Train loss: 8.426131248474121
	Test loss: 8.779203414916992
Epoch: 10
	Train loss: 8.410293579101562
	Test loss: 8.832548141479492
Epoch: 11
	Train loss: 8.393333435058594
	Test loss: 8.819811820983887
Epoch: 12
	Train loss: 8.377745628356934
	Test loss: 8.830615997314453
Epoch: 13
	Train loss: 8.36429214477539
	Test loss: 8.811063766479492
Epoch: 14
	Train loss: 8.350285530090332
	Test loss: 8.826931953430176
Epoch: 15
	Train l

### Experiment 3

In [22]:
wandb.login()
wandb.init(project="iad_homework1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


mean test loss,█▅▁▆▃▂▃▇▁▆▅▅▄▅▇
mean train loss,█▅▄▄▃▃▂▂▂▂▂▁▁▁▁
mean test loss,8.85021
mean train loss,8.33838


In [23]:
model3 = nn.Sequential(
    nn.Linear(90, 65),
    nn.ReLU(),
    nn.Linear(65, 40),
    nn.ReLU(),
    nn.Linear(40, 1)
)
wandb.watch(model3);

optimizer3 = torch.optim.SGD(model3.parameters(), lr=0.01)

In [24]:
train_norm(model3, optimizer3, criterion, train_loader_norm, test_loader_norm, epoch)

Epoch: 1
	Train loss: 9.67616081237793
	Test loss: 9.20719051361084
Epoch: 2
	Train loss: 9.069958686828613
	Test loss: 9.055336952209473
Epoch: 3
	Train loss: 8.94146728515625
	Test loss: 8.990891456604004
Epoch: 4
	Train loss: 8.862330436706543
	Test loss: 8.950298309326172
Epoch: 5
	Train loss: 8.806878089904785
	Test loss: 8.91841983795166
Epoch: 6
	Train loss: 8.767107009887695
	Test loss: 8.898680686950684
Epoch: 7
	Train loss: 8.73416519165039
	Test loss: 8.880474090576172
Epoch: 8
	Train loss: 8.705912590026855
	Test loss: 8.880668640136719
Epoch: 9
	Train loss: 8.681490898132324
	Test loss: 8.86868953704834
Epoch: 10
	Train loss: 8.65871524810791
	Test loss: 8.846396446228027
Epoch: 11
	Train loss: 8.642598152160645
	Test loss: 8.849329948425293
Epoch: 12
	Train loss: 8.625697135925293
	Test loss: 8.856740951538086
Epoch: 13
	Train loss: 8.607739448547363
	Test loss: 8.851648330688477
Epoch: 14
	Train loss: 8.59493350982666
	Test loss: 8.84288215637207
Epoch: 15
	Train loss: 8

### Experiment 4

In [25]:
wandb.login()
wandb.init(project="iad_homework1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


mean test loss,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁
mean train loss,█▄▃▃▂▂▂▂▂▁▁▁▁▁▁
mean test loss,8.83279
mean train loss,8.58078


In [26]:
model4 = nn.Sequential(
    nn.Linear(90, 65),
    nn.ReLU(),
    nn.Linear(65, 40),
    nn.ReLU(),
    nn.Linear(40, 1)
)
wandb.watch(model4);

optimizer4 = torch.optim.SGD(model4.parameters(), lr=0.01, momentum=0.9)

In [27]:
train_norm(model4, optimizer4, criterion, train_loader_norm, test_loader_norm, epoch)

Epoch: 1
	Train loss: 9.024303436279297
	Test loss: 8.896369934082031
Epoch: 2
	Train loss: 8.73499870300293
	Test loss: 8.81856918334961
Epoch: 3
	Train loss: 8.66118049621582
	Test loss: 8.796483039855957
Epoch: 4
	Train loss: 8.608110427856445
	Test loss: 8.795014381408691
Epoch: 5
	Train loss: 8.570029258728027
	Test loss: 8.856142044067383
Epoch: 6
	Train loss: 8.538188934326172
	Test loss: 8.803112030029297
Epoch: 7
	Train loss: 8.51578140258789
	Test loss: 8.797987937927246
Epoch: 8
	Train loss: 8.490795135498047
	Test loss: 8.793639183044434
Epoch: 9
	Train loss: 8.474849700927734
	Test loss: 8.832561492919922
Epoch: 10
	Train loss: 8.458909034729004
	Test loss: 8.819098472595215
Epoch: 11
	Train loss: 8.437989234924316
	Test loss: 8.911458015441895
Epoch: 12
	Train loss: 8.424200057983398
	Test loss: 8.813583374023438
Epoch: 13
	Train loss: 8.41938304901123
	Test loss: 8.805353164672852
Epoch: 14
	Train loss: 8.400432586669922
	Test loss: 8.815496444702148
Epoch: 15
	Train los

### Experiment 5

In [28]:
wandb.login()
wandb.init(project="iad_homework1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


mean test loss,▇▂▁▁▅▂▁▁▃▃█▂▂▂▃
mean train loss,█▅▄▃▃▃▂▂▂▂▂▁▁▁▁
mean test loss,8.82966
mean train loss,8.38472


In [29]:
model5 = nn.Sequential(
    nn.Linear(90, 65),
    nn.ReLU(),
    nn.Linear(65, 40),
    nn.ReLU(),
    nn.Linear(40, 1)
)
wandb.watch(model5);

optimizer5 = torch.optim.Adam(model5.parameters(), lr=0.0003, weight_decay=0.0005)

In [30]:
train_norm(model5, optimizer5, criterion, train_loader_norm, test_loader_norm, epoch, save=True)

Epoch: 1
	Train loss: 9.193089485168457
	Test loss: 8.959146499633789
Epoch: 2
	Train loss: 8.800749778747559
	Test loss: 8.862247467041016
Epoch: 3
	Train loss: 8.704700469970703
	Test loss: 8.829577445983887
Epoch: 4
	Train loss: 8.647235870361328
	Test loss: 8.824870109558105
Epoch: 5
	Train loss: 8.60657024383545
	Test loss: 8.813486099243164
Epoch: 6
	Train loss: 8.574443817138672
	Test loss: 8.803526878356934
Epoch: 7
	Train loss: 8.547737121582031
	Test loss: 8.791584014892578
Epoch: 8
	Train loss: 8.525853157043457
	Test loss: 8.792210578918457
Epoch: 9
	Train loss: 8.50788688659668
	Test loss: 8.774969100952148
Epoch: 10
	Train loss: 8.489452362060547
	Test loss: 8.803174018859863
Epoch: 11
	Train loss: 8.475903511047363
	Test loss: 8.771638870239258
Epoch: 12
	Train loss: 8.464052200317383
	Test loss: 8.797198295593262
Epoch: 13
	Train loss: 8.454398155212402
	Test loss: 8.77281665802002
Epoch: 14
	Train loss: 8.444628715515137
	Test loss: 8.791716575622559
Epoch: 15
	Train l

## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

**Отчет о проделанных экспериментах**: https://wandb.ai/dgkomarov_hse/iad_homework1/reports/-DL-1--VmlldzoyNzM5Nzcx?accessToken=7i2n103538yc4ne3wfjl4w21wlslecbcivt2s62v40oysmcun150sf8qlgyc24z4

**Итоговая модель и RMSE на тестовой выборке**:

In [31]:
model = nn.Sequential(
    nn.Linear(90, 65),
    nn.ReLU(),
    nn.Linear(65, 40),
    nn.ReLU(),
    nn.Linear(40, 1)
)
model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [32]:
print(f'RMSE Test: {test_norm(model, criterion, test_loader_norm)["RMSE"]}')

RMSE Test: 8.775067329406738


**Лучшая модель с весами**: https://drive.google.com/file/d/1bSjl0NDrM5Rg4d0og6hRG9B_Bjt-FSZd/view?usp=sharing